<a href="https://colab.research.google.com/github/FelipeRuanMarinheiro/Portifolio/blob/main/Regressions_OLS_FE_RE_Econometrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dados em Painel
- Modelo POLS
- Efeito Fixo (FE)
- Efeito Aleatorio (RE)
- Teste F de Chow
- Breausch-Pagen


##LIBS

In [129]:
!pip install matplotlib
!pip install pandas
!pip install linearmodels


import pandas as pd
import matplotlib.pyplot as plt

from linearmodels.panel import PanelOLS


##EXPLORATORIA

In [130]:
df = pd.read_csv('mortes.csv', sep=';')
df

,UF,id,mortalidade,renda,educacao,ano
0,RO,11,"0,186544",11312,287,2005
1,RO,11,"0,206278",11507,313,2006
2,RO,11,"0,208791",13005,409,2007
3,RO,11,"0,264931",13059,473,2008
4,RO,11,"0,254986",13917,516,2009
...,...,...,...,...,...,...
130,DF,53,"0,176999",51122,23964,2005
131,DF,53,"0,173059",52759,27678,2006
132,DF,53,"0,198859",54227,30762,2007
133,DF,53,"0,166238",54058,35312,2008


In [131]:
print(df.dtypes)



UF             object
id              int64
mortalidade    object
renda           int64
educacao        int64
ano             int64
dtype: object


In [132]:
# Substituir vírgulas por pontos na coluna 'mortalidade'
df['mortalidade'] = df['mortalidade'].str.replace(',', '.')

# Converter a coluna 'mortalidade' para float
df['mortalidade'] = df['mortalidade'].astype(float)

print(df.dtypes)


UF              object
id               int64
mortalidade    float64
renda            int64
educacao         int64
ano              int64
dtype: object


In [133]:
df.head()

,UF,id,mortalidade,renda,educacao,ano
0,RO,11,0.186544,11312,287,2005
1,RO,11,0.206278,11507,313,2006
2,RO,11,0.208791,13005,409,2007
3,RO,11,0.264931,13059,473,2008
4,RO,11,0.254986,13917,516,2009


##POLS

In [134]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

model_pols = smf.ols('mortalidade ~ renda + educacao', data=df).fit()
print(model_pols.summary())


                            OLS Regression Results                            
Dep. Variable:            mortalidade   R-squared:                       0.129
Model:                            OLS   Adj. R-squared:                  0.116
Method:                 Least Squares   F-statistic:                     9.779
Date:                Fri, 31 May 2024   Prob (F-statistic):           0.000110
Time:                        02:30:22   Log-Likelihood:                 191.25
No. Observations:                 135   AIC:                            -376.5
Df Residuals:                     132   BIC:                            -367.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.1406      0.013     10.683      0.0

##Efeito FIXO

In [135]:
model_efeitos_fixos = smf.ols('mortalidade ~ renda + educacao + C(UF)', data=df).fit()
print(model_efeitos_fixos.summary())


                            OLS Regression Results                            
Dep. Variable:            mortalidade   R-squared:                       0.939
Model:                            OLS   Adj. R-squared:                  0.923
Method:                 Least Squares   F-statistic:                     58.09
Date:                Fri, 31 May 2024   Prob (F-statistic):           4.50e-52
Time:                        02:30:22   Log-Likelihood:                 370.51
No. Observations:                 135   AIC:                            -683.0
Df Residuals:                     106   BIC:                            -598.8
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.0664      0.025      2.676      

##Efeito aleatorio

In [136]:
model_efeitos_aleatorios = smf.mixedlm('mortalidade ~ renda + educacao', data=df, groups=df['UF']).fit()
print(model_efeitos_aleatorios.summary())


          Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: mortalidade
No. Observations: 135     Method:             REML       
No. Groups:       27      Scale:              0.0003     
Min. group size:  5       Log-Likelihood:     270.4333   
Max. group size:  5       Converged:          Yes        
Mean group size:  5.0                                    
----------------------------------------------------------
           Coef.   Std.Err.    z     P>|z|  [0.025  0.975]
----------------------------------------------------------
Intercept   0.139     0.022   6.276  0.000   0.096   0.183
renda       0.000     0.000   2.864  0.004   0.000   0.000
educacao   -0.000     0.000  -2.336  0.019  -0.000  -0.000
Group Var   0.004     0.068                               



/usr/local/lib/python3.10/dist-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


## Teste F de Chow (POLS vs Efeito fixo)

In [137]:
from statsmodels.stats.anova import anova_lm

anova_results = anova_lm(model_pols, model_efeitos_fixos)
print(anova_results)

   df_resid       ssr  df_diff   ss_diff          F        Pr(>F)
0     132.0  0.464874      0.0       NaN        NaN           NaN
1     106.0  0.032658     26.0  0.432216  53.956526  1.274873e-49


## Teste de Breusch-Pagan (Heterocedasticidade)

In [138]:
from statsmodels.stats.diagnostic import het_breuschpagan

_, pvalue, _, _ = het_breuschpagan(model_efeitos_fixos.resid, model_efeitos_fixos.model.exog)
print("Teste de Breusch-Pagan (Efeitos Fixos): p-valor =", pvalue)

_, pvalue, _, _ = het_breuschpagan(model_efeitos_aleatorios.resid, model_efeitos_aleatorios.model.exog)
print("Teste de Breusch-Pagan (Efeitos Aleatórios): p-valor =", pvalue)


Teste de Breusch-Pagan (Efeitos Fixos): p-valor = 0.032626981994788154
Teste de Breusch-Pagan (Efeitos Aleatórios): p-valor = 0.48638044077260245


Desta forma, escolhe-se o modelo aleatorio haja vista que os dados são homocedasticos em relação ao modelo fixo.